# Pilot-Quantum 

In [2]:
import os, time, sys
sys.path.insert(0, os.path.abspath('../../..'))
import socket
import distributed, dask
import numpy as np
import pandas as pd
import dask.dataframe as dd
import dask.array as da
import dask.bag as db
import pilot.streaming
from qiskit_benchmark import run_graph, generate_data
from qiskit.primitives import Estimator
import getpass
import datetime

## Start Pilot

In [11]:
RESOURCE_URL_HPC = "slurm://localhost"
WORKING_DIRECTORY = os.path.join(os.environ["PSCRATCH"], "work")

def start_pilot(number_cores):
    pilot_compute_description_dask = {
        #"resource": "ssh://{}@localhost".format(getpass.getuser()),
        "resource": RESOURCE_URL_HPC,
        #"working_directory": os.path.join(os.path.expanduser("~"), "work"),
        "working_directory": WORKING_DIRECTORY,
        #"number_of_nodes": 1,
        "number_cores": number_cores,
        #"cores_per_node": number_cores,
        "queue": "normal",
        "walltime": 5,
        "type": "dask",
        "project": "m4408",
        "os_ssh_keyfile": "~/.ssh/nersc",
        "scheduler_script_commands": ["#SBATCH --constraint=cpu"]
    }

#     pilot_compute_description_dask = {
#     "resource": RESOURCE_URL_HPC,
#     "working_directory": WORKING_DIRECTORY,
#     "number_cores": 1,
#     "queue": "normal",
#     "walltime": 5,
#     "type": "dask",
#     "project": "m4408",
#     "os_ssh_keyfile": "~/.ssh/nersc",
#     "scheduler_script_commands": ["#SBATCH --constraint=cpu"]
# }


    dask_pilot = pilot.streaming.PilotComputeService.create_pilot(pilot_compute_description_dask)
    dask_pilot.wait()
    dask_pilot.get_details()
    dask_client  = distributed.Client(dask_pilot.get_details()['master_url'])
    return dask_pilot, dask_client

In [ ]:
dask_pilot, dask_client = start_pilot(1)
dask_client.scheduler_info()

In [ ]:
dask_client.gather(dask_client.map(lambda a: a*a, range(10)))

## Test different quantum workloads with Pilot

In [ ]:
run_graph(depth_of_recursion=1,
        num_qubits=2,
        n_entries=1,
        circuit_depth=1,
        size_of_observable=1,
        n_backends=1)


In [ ]:
circuits, observables =  generate_data(
            depth_of_recursion=1, # number of circuits and observables
            num_qubits=1,
            n_entries=3, # number of circuits and observables => same as depth_of_recursion
            circuit_depth=1,
            size_of_observable=1
        )

In [ ]:
circuits[0].draw()

In [ ]:

observables[0]

In [ ]:
Estimator().run(circuits, observables).result()

In [ ]:
# create array of tuples with circuits and collections
circuits_observables =zip(circuits, observables)
circuit_bag = db.from_sequence(circuits_observables) 

In [ ]:
circuit_bag.map(lambda circ_obs: Estimator().run(circ_obs[0], circ_obs[1]).result()).compute()

In [ ]:
dask_pilot.cancel()

## Micro Benchmark 

In [13]:
# create a benchmark loop
num_qubits = 2
n_entries = 1024
results = []
run_timestamp=datetime.datetime.now()
RESULT_FILE= "pilot-quantum-summary-" + run_timestamp.strftime("%Y%m%d-%H%M%S") + ".csv"
       
for number_cores in [1]:
    start = time.time()
    dask_pilot, dask_client = start_pilot(number_cores)
    circuits, observables =  generate_data(
            depth_of_recursion=1, # number of circuits and observables
            num_qubits=num_qubits,
            n_entries=n_entries, # number of circuits and observables => same as depth_of_recursion
            circuit_depth=1,
            size_of_observable=1
    ) 
    end_pilot = time.time()
    circuits_observables =zip(circuits, observables)
    circuit_bag = db.from_sequence(circuits_observables)
    circuit_bag.map(lambda circ_obs: Estimator().run(circ_obs[0], circ_obs[1]).result()).compute()
    end_compute = time.time()
    dask_pilot.cancel()
    end_stop_pilot = time.time()
    # write data to file
    result_string = "Number Cores: {} Number Qubits: {} Number Circuits {} Pilot start: {}, Total: {}, Compute: {}s".format(number_cores, num_qubits, n_entries, end_pilot-start, end_stop_pilot-start, end_compute-end_pilot)
    results.append(result_string)
    with open(RESULT_FILE, "w") as f:
        f.write(results)
    time.sleep(60)

08/30/2023 02:00:55 PM - root - DEBUG - Run python Args: ['-m', 'pilot.plugins.dask.bootstrap_dask', ' -p ', '1']
08/30/2023 02:00:55 PM - root - DEBUG - Command: python -m pilot.plugins.dask.bootstrap_dask  -p  1
08/30/2023 02:00:55 PM - pilot-streaming - DEBUG - Pilot-Streaming SLURM: Parsing job description: {'executable': 'python', 'arguments': ['-m', 'pilot.plugins.dask.bootstrap_dask', ' -p ', '1'], 'working_directory': '/pscratch/sd/l/luckow/work/dask-5008a21e-4778-11ee-9e64-93c1fd62f0b9', 'output': 'dask_job_dask-5008a21e-4778-11ee-9e64-93c1fd62f0b9.stdout', 'error': 'dask_job_dask-5008a21e-4778-11ee-9e64-93c1fd62f0b9.stderr', 'number_of_nodes': 1, 'cores_per_node': 1, 'project': 'm4408', 'reservation': None, 'queue': 'normal', 'walltime': 5, 'pilot_compute_description': {'resource': 'slurm://localhost', 'working_directory': '/pscratch/sd/l/luckow/work', 'number_cores': 1, 'queue': 'normal', 'walltime': 5, 'type': 'dask', 'project': 'm4408', 'os_ssh_keyfile': '~/.ssh/nersc', 's

Working Directory: /pscratch/sd/l/luckow/work
dask-5008a21e-4778-11ee-9e64-93c1fd62f0b9/pscratch/sd/l/luckow/work
/tmp/tmpugr1sk9v


***************************************************************************
                          NOTICE TO USERS

Lawrence Berkeley National Laboratory operates this computer system under 
contract to the U.S. Department of Energy.  This computer system is the 
property of the United States Government and is for authorized use only.
Users (authorized or unauthorized) have no explicit or implicit 
expectation of privacy.

Any or all uses of this system and all files on this system may be
intercepted, monitored, recorded, copied, audited, inspected, and disclosed
to authorized site, Department of Energy, and law enforcement personnel,
as well as authorized officials of other agencies, both domestic and foreign.
By using this system, the user consents to such interception, monitoring,
recording, copying, auditing, inspection, and disclosure at the discretion
of authorized site or Department of Energy personnel.

Unauthorized or improper use of this system may result in administrative

Submission of Job Command: ssh localhost sbatch  tmpugr1sk9v


2023-08-30 14:01:04,395 - distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client


...

{'type': 'Scheduler', 'id': 'Scheduler-5a5896b1-472d-40fa-8939-ecab712b3d59', 'address': 'tcp://128.55.71.47:8786', 'services': {'dashboard': 8787}, 'started': 1693429352.0531318, 'workers': {}}


08/30/2023 02:03:03 PM - pilot-streaming - DEBUG - Cancel SLURM job
08/30/2023 02:03:06 PM - root - DEBUG - Cancel SLURM job: scancel 14684560 Output: 
08/30/2023 02:03:06 PM - root - DEBUG - No job with ID 14684560 found
08/30/2023 02:03:06 PM - root - DEBUG - Results of scheduler startup file check: True
08/30/2023 02:03:06 PM - asyncio - DEBUG - Using selector: EpollSelector


Connect to Dask: tcp://nid004296:8786


2023-08-30 14:03:36,248 - distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client
2023-08-30 14:03:36,277 - distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client


type: [Errno 2] No such file or directory: 'results/pilot-qunatum-summary-20230830-140055.csv'

In [ ]:
print(results)